In [ ]:
# 
# Build pLTV using GAP bigquery and Lifetimes ML library 
# Used calibration period for training & holdout period for validation without considering cohort transactions
# Script can be run on Compute Engine of GCP
#
# THIS CODE AND INFORMATION ARE PROVIDED "AS IS" WITHOUT WARRANTY OF ANY KIND
#
# By JeeWook Kim
#

## install libraries

In [1]:
%%bash
pip install --upgrade pip
pip install --upgrade google-api-python-client
pip install --upgrade gcloud
pip install lifetimes

  Using cached https://files.pythonhosted.org/packages/0f/74/ecd13431bcc456ed390b44c8a6e917c1820365cbebcb6a8974d1cd045ab4/pip-10.0.1-py2.py3-none-any.whl
  Found existing installation: pip 9.0.1
    Uninstalling pip-9.0.1:
      Successfully uninstalled pip-9.0.1
  Using cached https://files.pythonhosted.org/packages/bd/40/bc3b4e7c7c65f9548024dde5c7bad60e0e078b2d2a0ee8c426a5639c2cc9/google_api_python_client-1.7.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/53/06/6e6d5bfa4d23ee40efd772d6b681a7afecd859a9176e564b8c329382370f/google_auth-1.5.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/33/49/c814d6d438b823441552198f096fcd0377fd6c88714dbed34f1d3c8c4389/google_auth_httplib2-0.0.3-py2.py3-none-any.whl
Requirement not upgraded as not directly required: six<2dev,>=1.6.1 in /usr/local/lib/python2.7/dist-packages (from google-api-python-client) (1.10.0)
Requirement not upgraded as not directly required: uritemplate<4dev,>=3.0.0 in /

google-cloud-resource-manager 0.26.0 has requirement google-cloud-core<0.27dev,>=0.26.0, but you'll have google-cloud-core 0.27.1 which is incompatible.
grpcio 1.6.3 has requirement protobuf>=3.3.0, but you'll have protobuf 3.2.0 which is incompatible.
google-cloud-error-reporting 0.26.0 has requirement google-cloud-core<0.27dev,>=0.26.0, but you'll have google-cloud-core 0.27.1 which is incompatible.
google-cloud-monitoring 0.26.0 has requirement google-cloud-core<0.27dev,>=0.26.0, but you'll have google-cloud-core 0.27.1 which is incompatible.
google-cloud 0.27.0 has requirement google-cloud-core<0.27dev,>=0.26.0, but you'll have google-cloud-core 0.27.1 which is incompatible.
google-cloud 0.27.0 has requirement google-cloud-storage<1.4dev,>=1.3.0, but you'll have google-cloud-storage 1.4.0 which is incompatible.
google-cloud-vision 0.26.0 has requirement google-cloud-core<0.27dev,>=0.26.0, but you'll have google-cloud-core 0.27.1 which is incompatible.
google-cloud-logging 1.2.0 has

## initiallize the values
####  Change the value of ldap variable to your ldap 
####  Change the value of cloud storage bucket name to your own bucket name

In [21]:
# Change the value of ldap variable to your ldap
ldap = 'jeewook'
bucket = 'tps-apac-ma'
import datetime

today = datetime.date.today().strftime("%Y%m%d")
# calibration begin date
begin_date = '20160801'
# obserbation end date
end_date = '20170801'
# calibration end date
calibration_end_date = '20170201'

# Googel Store demo table
# If you have your own GAP BQ trable, use yours
gap_table = 'bigquery-public-data.google_analytics_sample.ga_sessions_*'

print('# today: {}'.format(today))
print('# begin_date: {}'.format(begin_date))
print('# end_date: {}'.format(end_date))
print('# calibration_end_date: {}'.format(calibration_end_date))

# query to retrieve GAP exported BigQuery ecommerce tranactions (users with purchases in the calibration period)

sql_train = """ 
  SELECT
  date,
  obs_data.fullVisitorId as fullVisitorId,
  sum(totals.transactionRevenue) as transactionRevenue,
  sum(totals.transactions) as transactions
  FROM
  `"""+gap_table+"""` obs_data
  
  INNER JOIN 
     ( SELECT 
         fullVisitorId
       FROM `"""+gap_table+"""`
       WHERE
       (_TABLE_SUFFIX >= '"""+begin_date+"""' AND _TABLE_SUFFIX <= '"""+calibration_end_date+"""') AND
       totals.transactions IS NOT NULL
     ) cal_data ON cal_data.fullVisitorId = obs_data.fullVisitorId 
  WHERE
  (_TABLE_SUFFIX >= '"""+begin_date+"""' AND _TABLE_SUFFIX <= '"""+end_date+"""') AND
  totals.transactions IS NOT NULL
  GROUP BY 1,2
  ORDER BY date
"""

print ('# BigQuery SQL - training data')  
print (sql_train)


# today: 20180626
# begin_date: 20160801
# end_date: 20170801
# calibration_end_date: 20170201
# BigQuery SQL - training data
 
  SELECT
  date,
  obs_data.fullVisitorId as fullVisitorId,
  sum(totals.transactionRevenue) as transactionRevenue,
  sum(totals.transactions) as transactions
  FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*` obs_data
  
  INNER JOIN 
     ( SELECT 
         fullVisitorId
       FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
       WHERE
       (_TABLE_SUFFIX >= '20160801' AND _TABLE_SUFFIX <= '20170201') AND
       totals.transactions IS NOT NULL
     ) cal_data ON cal_data.fullVisitorId = obs_data.fullVisitorId 
  WHERE
  (_TABLE_SUFFIX >= '20160801' AND _TABLE_SUFFIX <= '20170801') AND
  totals.transactions IS NOT NULL
  GROUP BY 1,2
  ORDER BY date



## Query GAP BigQuery data and load into pandas dataframe

In [4]:
# execute the query using datalab bigquery lib
import google.datalab.bigquery as bq
# Pandas lib to handle table data
import pandas as pd
transaction_query = bq.Query(sql_train)
query_result = transaction_query.execute()

# copy the query result to pandas dataframe
transaction_query_data = query_result.result().to_dataframe()
transaction_query_data['transactionRevenue'] = transaction_query_data['transactionRevenue'] / 1000
transaction_query_data.head(50)

 

,date,fullVisitorId,transactionRevenue,transactions
0,20160801,5760753352577829144,47990.0,1
1,20160801,6332431001963915083,54380.0,2
2,20160801,6452285658468801695,31990.0,1
3,20160801,4993485206334150199,16990.0,1
4,20160801,2976178532750719771,15930.0,1
5,20160801,2509714289037323244,144190.0,1
6,20160801,0831511922986457407,61900.0,1
7,20160801,7589137567725941774,20780.0,1
8,20160801,9998597322098588317,97200.0,1
9,20160801,8844262168616364981,111910.0,1


## Transform the query result dataframe into calibraion and holdout RFM data to be used lifetimes model

In [5]:
# datalab kernel needs to be python2 to be compatible with Lifetimes lib
from lifetimes.utils import calibration_and_holdout_data
# transform the query result dataframe into calibraion and holdout RFM data to be used lifetimes model
summary_cal_holdout = calibration_and_holdout_data(transaction_query_data, 'fullVisitorId', 'date',
                                        calibration_period_end=calibration_end_date,
                                        observation_period_end=end_date, monetary_value_col='transactionRevenue')   
print('# summary_cal_holdout')
print(summary_cal_holdout.head(50))


# summary_cal_holdout
                     frequency_cal  recency_cal  T_cal  monetary_value_cal  \
fullVisitorId                                                                
0000677695778949032            0.0          0.0   69.0                 0.0   
0002841484597578555            0.0          0.0   84.0                 0.0   
0002871498069867123            0.0          0.0  162.0                 0.0   
0003450834640354121            0.0          0.0  106.0                 0.0   
000435324061339869             0.0          0.0  104.0                 0.0   
0006972685091967991            0.0          0.0   66.0                 0.0   
0007617910709180468            0.0          0.0   51.0                 0.0   
0008543502700911746            0.0          0.0   43.0                 0.0   
0012453214936159528            0.0          0.0   58.0                 0.0   
0016314221806046125            0.0          0.0   78.0                 0.0   
0018822472571637380            1.0        

## Train the lifetime BGF modle using the calibration data and save the model file

In [8]:
# train the lifetime BGF modle using the calibration data
from lifetimes.plotting import plot_calibration_purchases_vs_holdout_purchases
from lifetimes import BetaGeoFitter

# similar API to scikit-learn and lifelines.
bgf = BetaGeoFitter(penalizer_coef=0.0)
# adjust wheather to use returning users or all the users to train the model 
returned_summary_cal_holdout = summary_cal_holdout[summary_cal_holdout['frequency_cal']>=0]
bgf.fit(returned_summary_cal_holdout['frequency_cal'], returned_summary_cal_holdout['recency_cal'], returned_summary_cal_holdout['T_cal'])

print ('# BetaGeoFitter')
print(bgf)
# Save the BGF model to be used later
bgf.save_model('bgf_'+ldap+'_'+today+'.pkl')


# BetaGeoFitter
<lifetimes.BetaGeoFitter: fitted with 10115 subjects, a: 2.84, alpha: 12.26, b: 1.78, r: 0.08>


# Validate the model by MAE, MSE, Recall Rate, Precision Rate and Lifetimes provides graphs 

In [9]:
# Validate the model by comparing the predicted frequecy with real frequecy data of holdout period
import numpy as np
vaildate_summary = returned_summary_cal_holdout.copy()
duration_holdout = vaildate_summary.iloc[0]['duration_holdout']
print("# duration_holdout: {}".format(duration_holdout))
# predict expected number of transactions
vaildate_summary['p_frequency'] = \
          vaildate_summary.apply(lambda r: bgf.conditional_expected_number_of_purchases_up_to_time(duration_holdout, \
          r['frequency_cal'], r['recency_cal'], r['T_cal']), axis=1)

print ('# vaildate_summary - model_predictions - expected_number_of_purchases')
print vaildate_summary['p_frequency'].head(50)
print ('# vaildate_summary - frequency_holdout - real data')
print vaildate_summary['frequency_holdout'].head(50)
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(vaildate_summary['frequency_holdout'], vaildate_summary['p_frequency'])
print ("# mean absolute error regression loss: {}".format(mae))
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(vaildate_summary['frequency_holdout'], vaildate_summary['p_frequency'])
print ("# mean squared error regression loss: {}".format(mse))

# Validate the model by calculating recall rate and precision rate
# predict p alive
vaildate_summary['p_alive'] = \
          vaildate_summary.apply(lambda r: bgf.conditional_probability_alive(duration_holdout, \
          r['frequency_cal'], r['recency_cal'], r['T_cal']), axis=1)

print ('# vaildate_summary - p_alive')
print vaildate_summary.head(100)
 
# Recall rate
a_purchasers = 0
p_purchasers = 0
for index, row in vaildate_summary.iterrows():
  if row['frequency_cal'] > 1 : # calculate recall rate only using the users with frequency > 1
    if row['frequency_holdout'] > 0 :  # actual purchaser
        a_purchasers = a_purchasers + 1
        if row['p_frequency'] >= 0.8 :  # predicted purchaser
          p_purchasers = p_purchasers + 1
recall_rate = float(p_purchasers)/a_purchasers
print ("# recall rate: {:.2f}, toal actual purchasers: {}, predicted purchasers out of actual purchasers: {}".
       format(recall_rate, a_purchasers, p_purchasers))

# Precision rate
a_purchasers = 0
p_purchasers = 0
for index, row in vaildate_summary.iterrows():
  if row['frequency_cal'] > 1 : # calculate recall rate only using the users with frequency > 1
    if row['p_frequency'] >= 0.8: # predicted purchaser
        p_purchasers = p_purchasers + 1
        if row['frequency_holdout'] > 0 :  # actual purchaser
          a_purchasers = a_purchasers + 1
precision_rate = float(a_purchasers)/p_purchasers
print ("# precision_rate: {:.2f}, total predicted purchasers: {}, actual purchasers out of predicted purchasers: {}".
       format(precision_rate, p_purchasers, a_purchasers))

import matplotlib.pyplot as plt
# Validate the model using lifetimes provided plots
#ax = plot_calibration_purchases_vs_holdout_purchases(bgf, summary_cal_holdout)
ax = plot_calibration_purchases_vs_holdout_purchases(bgf, returned_summary_cal_holdout)
ax.figure.savefig('calibration_purchases_vs_holdout_purchases_'+ldap+'_'+today+'.png')
plt.close(ax.figure)
# Plot predicted purchases vs. actual purchases in calibration period
from lifetimes.plotting import plot_period_transactions
ax2 = plot_period_transactions(bgf)
ax2.figure.savefig('plot_period_transactions_'+ldap+'_'+today+'.png')
plt.close(ax2.figure)
from lifetimes.plotting import plot_frequency_recency_matrix
ax3 = plot_frequency_recency_matrix(bgf)
ax3.figure.savefig('plot_frequency_recency_matrix_'+ldap+'_'+today+'.png')
plt.close(ax3.figure)
from lifetimes.plotting import plot_probability_alive_matrix
ax4 = plot_probability_alive_matrix(bgf)
ax4.figure.savefig('plot_probability_alive_matrix_'+ldap+'_'+today+'.png')
plt.close(ax4.figure)


# duration_holdout: 181.0
# vaildate_summary - model_predictions - expected_number_of_purchases
fullVisitorId
0000677695778949032    0.103840
0002841484597578555    0.092514
0002871498069867123    0.059592
0003450834640354121    0.079909
000435324061339869     0.080905
0006972685091967991    0.106470
0007617910709180468    0.122122
0008543502700911746    0.132731
0012453214936159528    0.114240
0016314221806046125    0.096716
0018822472571637380    0.233785
001905118576359487     0.079420
0019225693815886027    0.256407
0019959816835777825    0.261476
0020342483972052451    0.115298
0020502892485275044    0.065182
002141771093125260     0.122122
0021430049270067403    0.468740
0021939395292136520    0.066837
0023136350593577983    0.168835
0024220924715322988    0.123346
0024266704681267535    0.244352
0024314485741511650    0.143852
0024725579067754075    0.161693
0025977414477335327    0.395486
0026203741366904270    0.063918
0027515094210676639    0.448584
0027563191767325777    0.2

## Query whole preriod of data to use as prediction data

In [10]:
# query to retrieve all the customers who purchases at least once
sql_predict = """ 
  SELECT
  date,
  fullVisitorId,
  sum(totals.transactionRevenue) as transactionRevenue,
  sum(totals.transactions) as transactions
  FROM
  `"""+gap_table+"""`
  WHERE
  (_TABLE_SUFFIX >= '"""+begin_date+"""' AND _TABLE_SUFFIX <= '"""+end_date+"""') AND
  totals.transactions IS NOT NULL
  GROUP BY 1,2
  ORDER BY date
  
"""
print ('# BigQuery SQL - predict')  
print (sql_predict)
transaction_query = bq.Query(sql_predict)
query_result = transaction_query.execute()

# copy the query result to pandas dataframe
transaction_query_data = query_result.result().to_dataframe()
transaction_query_data['transactionRevenue'] = transaction_query_data['transactionRevenue'] / 1000
transaction_query_data.head(50)


# BigQuery SQL - predict
 
  SELECT
  date,
  fullVisitorId,
  sum(totals.transactionRevenue) as transactionRevenue,
  sum(totals.transactions) as transactions
  FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
  WHERE
  (_TABLE_SUFFIX >= '20160801' AND _TABLE_SUFFIX <= '20170801') AND
  totals.transactions IS NOT NULL
  GROUP BY 1,2
  ORDER BY date
  



,date,fullVisitorId,transactionRevenue,transactions
0,20160801,280571682929338005,8990.0,1
1,20160801,2509714289037323244,144190.0,1
2,20160801,1817256435818850257,3500.0,1
3,20160801,3006489767948351126,2211380.0,1
4,20160801,7589137567725941774,20780.0,1
5,20160801,6027268712782791947,27180.0,1
6,20160801,2976178532750719771,15930.0,1
7,20160801,7545791097611675575,102350.0,1
8,20160801,9998597322098588317,97200.0,1
9,20160801,2125540555068339394,36780.0,1


## Transform the query result dataframe into RFM (use the whole period of data)

In [11]:
# Transform the query result dataframe into RFM (use whole period of data)
from lifetimes.utils import summary_data_from_transaction_data
summary = \
  summary_data_from_transaction_data(transaction_query_data, 'fullVisitorId', 'date', \
  observation_period_end=end_date, monetary_value_col='transactionRevenue')
print ('# summary_data_from_transaction_data')
print(summary.head(50))


# summary_data_from_transaction_data
                     frequency  recency      T  monetary_value
fullVisitorId                                                 
0000213131142648941        0.0      0.0   95.0             0.0
0000677695778949032        1.0    107.0  250.0        268000.0
0002841484597578555        0.0      0.0  265.0             0.0
0002871498069867123        0.0      0.0  343.0             0.0
0003450834640354121        0.0      0.0  287.0             0.0
0003961110741104601        0.0      0.0   72.0             0.0
0004186385457250117        1.0     11.0  173.0        119000.0
000435324061339869         0.0      0.0  285.0             0.0
0004478134742292937        0.0      0.0    3.0             0.0
0006972685091967991        0.0      0.0  247.0             0.0
0007177934947634478        0.0      0.0    7.0             0.0
0007617910709180468        0.0      0.0  232.0             0.0
0007933257389091624        0.0      0.0   92.0             0.0
000854350270091174

## Predict expected number of purchases and probability of alive using returning customer data 

In [13]:
# Prepare returning customers data to be used in GGF model, GGF training works with customers with monetary value
returning_customers_summary = summary[(summary['frequency']>0) & (summary['monetary_value']>0)]
print ('# returning_customers_summary')
print(returning_customers_summary.head())

# Predict predicted_purchases using the trained BGF model
t = 181 # 181 time units
returning_customers_summary['predicted_purchases'] = \
  bgf.conditional_expected_number_of_purchases_up_to_time(
    t,
    returning_customers_summary['frequency'],
    returning_customers_summary['recency'], 
    returning_customers_summary['T']
  )
returning_customers_summary['p_alive'] = \
  bgf.conditional_probability_alive(
    t, 
    returning_customers_summary['frequency'], 
    returning_customers_summary['recency'], 
    returning_customers_summary['T']
  )
returning_customers_summary.sort_values(by='predicted_purchases',ascending=False).head(50)


# returning_customers_summary
                     frequency  recency      T  monetary_value
fullVisitorId                                                 
0000677695778949032        1.0    107.0  250.0        268000.0
0004186385457250117        1.0     11.0  173.0        119000.0
0012549450274915791        1.0      2.0  175.0        149000.0
0014262055593378383        2.0     18.0   35.0         16585.0
0016780404116867280        1.0     50.0   96.0        238000.0


,frequency,recency,T,monetary_value,predicted_purchases,p_alive
fullVisitorId,,,,,,
9077649148974613765,7.0,165.0,183.0,1.419886e+05,1.942453,0.000000
4984366501121503466,14.0,251.0,293.0,5.689257e+05,1.913985,0.000000
8197879643797712877,9.0,243.0,273.0,4.299467e+05,1.892119,0.000000
1957458976293878100,13.0,136.0,168.0,4.558302e+06,1.618845,0.000000
7463172420271311409,8.0,252.0,292.0,8.271838e+05,1.466687,0.000000
9681060687378784629,3.0,11.0,15.0,4.112913e+06,1.456976,0.000000
2610708601231231422,5.0,105.0,125.0,1.145020e+05,1.430115,0.000000
910504852442385120,6.0,257.0,284.0,2.727583e+05,1.330755,0.000000
8015915032010696677,2.0,11.0,12.0,1.535000e+05,1.249354,0.000000


## Predict expected average profit and customer lifetime value

In [14]:
# Train GGF model using the returning customers data
from lifetimes import GammaGammaFitter

ggf = GammaGammaFitter(penalizer_coef = 0)
ggf.fit(returning_customers_summary['frequency'],
        returning_customers_summary['monetary_value'])
print ('# GammaGammaFitter')
print(ggf)

# Save the GGF model to be used later
ggf.save_model('ggf_'+ldap+'_'+today+'.pkl')

# Predict expected_average_profit using the trained GGF model
returning_customers_summary['expected_average_profit'] =  ggf.conditional_expected_average_profit(
        returning_customers_summary['frequency'],
        returning_customers_summary['monetary_value']
    )
returning_customers_summary.sort_values(by='expected_average_profit',ascending=False).head(50)

# Predict customer_lifetime_value using the trained GGF model and BGF model

returning_customers_summary['customer_lifetime_value'] = ggf.customer_lifetime_value(
    bgf, # the trained model to use to predict the number of future transactions
    returning_customers_summary['frequency'],
    returning_customers_summary['recency'],
    returning_customers_summary['T'],
    returning_customers_summary['monetary_value'],
    time=12, # months
    discount_rate=0.01)
returning_customers_summary.sort_values(by='customer_lifetime_value',ascending=False).head(50)
print ('# returning_customers_summary')
print returning_customers_summary

# GammaGammaFitter
<lifetimes.GammaGammaFitter: fitted with 2583 subjects, p: 1.71, q: 4.09, v: 372068.08>
# returning_customers_summary
                     frequency  recency      T  monetary_value  \
fullVisitorId                                                    
0000677695778949032        1.0    107.0  250.0        268000.0   
0004186385457250117        1.0     11.0  173.0        119000.0   
0012549450274915791        1.0      2.0  175.0        149000.0   
0014262055593378383        2.0     18.0   35.0         16585.0   
0016780404116867280        1.0     50.0   96.0        238000.0   
0018822472571637380        1.0     34.0  265.0        238000.0   
0019225693815886027        1.0      5.0  216.0        119000.0   
0021430049270067403        2.0     23.0  232.0        298000.0   
0024266704681267535        1.0     27.0  253.0        119000.0   
0025977414477335327        1.0     67.0  257.0        119000.0   
0027515094210676639        1.0     46.0  235.0        506000.0   
00364

## Store the predicted output to cloud storage and BigQuery to be used to integrate back to GAP

In [20]:
# fill na with 0
returning_customers_summary.fillna(value=0, inplace=True)
returning_customers_summary.dtypes

# Store output to cloud storage
import datalab.storage as gcs
gcs.Bucket(bucket).item('pltv_demo_output_'+ldap+'_'+today+'.csv').write_to(returning_customers_summary.to_csv(),'text/csv')

# Store output to BQ table
summary_with_index_column = returning_customers_summary.reset_index()
print ('# summary_with_index_column')
print summary_with_index_column
schema = bq.Schema.from_data(summary_with_index_column)
print ('# BQ schema')
print schema
table = ('animals-in-space-app-35426', 'tps_apac_ma_data', 'pltv_demo_output_'+ldap+'_'+today)
output_table = bq.Table(table)
output_table.create(schema,overwrite=True)

output_table.insert(summary_with_index_column)




# summary_with_index_column
            fullVisitorId  frequency  recency      T  monetary_value  \
0     0000677695778949032        1.0    107.0  250.0        268000.0   
1     0004186385457250117        1.0     11.0  173.0        119000.0   
2     0012549450274915791        1.0      2.0  175.0        149000.0   
3     0014262055593378383        2.0     18.0   35.0         16585.0   
4     0016780404116867280        1.0     50.0   96.0        238000.0   
5     0018822472571637380        1.0     34.0  265.0        238000.0   
6     0019225693815886027        1.0      5.0  216.0        119000.0   
7     0021430049270067403        2.0     23.0  232.0        298000.0   
8     0024266704681267535        1.0     27.0  253.0        119000.0   
9     0025977414477335327        1.0     67.0  257.0        119000.0   
10    0027515094210676639        1.0     46.0  235.0        506000.0   
11    0036417634769000138        1.0     11.0  306.0         91980.0   
12    0039541267107154262        1.0

fullVisitorId,frequency,recency,T,monetary_value,predicted_purchases,p_alive,expected_average_profit,customer_lifetime_value
6074571277772704163,1.0,6.0,125.0,237000.0,0.0573170686299,4.33293243268e-24,216796.792626,17203.9310592
8616386948005204197,2.0,17.0,244.0,253495.0,0.0107554994285,1.85839445125e-55,230760.477432,3581.09960788
9925221725724431549,1.0,17.0,196.0,119000.0,0.0459019855462,3.54384445132e-61,174794.884388,11782.4887486
0937521038759977999,1.0,23.0,57.0,132550.0,0.289624689597,0.0,179617.984867,65405.2214007
817007348095653854,1.0,58.0,309.0,115800.0,0.0512632515936,4.515749654e-130,173655.849588,13831.5437849
4531247843823782323,1.0,30.0,139.0,522050.0,0.110693680213,0.0,318259.876891,49459.2611211
8101107214208089371,3.0,61.0,122.0,133980.0,0.348409073444,0.0,160936.64388,70494.4976051
9503612993189278427,1.0,79.0,168.0,33590.0,0.16641030612,0.0,144393.333687,34584.8782411
1494014144412248651,1.0,1.0,351.0,55550.0,0.007479977274,0.984623984045,152209.96,1795.00684305
5656077731099791990,1.0,14.0,183.0,149000.0,0.0459632573247,1.1423809666e-52,185473.335635,12408.0377951
